In [15]:
!pip install fastapi
!pip install uvicorn
!pip install pickle5
!pip install pydantic
!pip install requests
!pip install pypi-json
!pip install pyngrok
!pip install nest-asyncio
!pip install diffusers
!pip install torchvision
!pip install datasets
!pip install accelerate
!pip install diffusers
!pip install tensorflow

In [2]:
!unzip ziped

Archive:  ziped.zip
   creating: ziped/
  inflating: ziped/model_index.json  
   creating: ziped/scheduler/
  inflating: ziped/scheduler/scheduler_config.json  
   creating: ziped/unet/
  inflating: ziped/unet/config.json  
  inflating: ziped/unet/diffusion_pytorch_model.safetensors  


In [9]:
import torch
from diffusers import DDPMPipeline, DDPMScheduler, UNet2DModel
from PIL import Image
import numpy as np
import os
import json
import random
from fastapi import FastAPI
from pydantic import BaseModel
import pickle
import json
import uvicorn
from pyngrok import ngrok
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import base64

# Define your color mapping
TILE_COLORS = {
    'grass': (0, 128, 0),    # green
    'path-level0': (165, 42, 42),   # brown
    'path_level1': (210, 180, 140),      # Path level 1
    'path_level2': (160, 82, 45),   # Path level 2
    'path_level3': (139, 69, 19),  # Path level 3
    'water': (0, 0, 255),    # blue
    'forest': (0, 100, 0),   # dark green
    'sand': (255, 255, 0),    # yellow
    'bridge': (255,150,0),
    'rocklevel1': (128,128,128),
    'rocklevel2': (169,169,169),
    'rocklevel3': (211,211,211)
}

def map_to_nearest_color(image):
    image_np = np.array(image)
    output = np.zeros_like(image_np)

    colors = np.array(list(TILE_COLORS.values()))
    distances = np.sum((image_np[:,:,None,:] - colors[None,None,:,:])**2, axis=3)
    nearest_color_indices = np.argmin(distances, axis=2)

    for i, color in enumerate(colors):
        output[nearest_color_indices == i] = color

    return Image.fromarray(output.astype(np.uint8))

def load_pipeline(base_path):
    # Load the model
    model = UNet2DModel.from_pretrained(base_path, subfolder="")

    # Load the scheduler
    with open(os.path.join("./ziped/scheduler", "scheduler_config.json"), "r") as f:
        scheduler_config = json.load(f)
    scheduler = DDPMScheduler.from_config(scheduler_config)

    # Create the pipeline
    pipeline = DDPMPipeline(unet=model, scheduler=scheduler)

    return pipeline

def generate_single_image(base_path, output_path, seed=42):
    # Load the pipeline
    pipeline = load_pipeline(base_path)
    pipeline = pipeline.to("cuda" if torch.cuda.is_available() else "cpu")

    # Generate the image
    generator = torch.Generator(device=pipeline.device).manual_seed(seed)
    image = pipeline(
        batch_size=1,
        generator=generator,
    ).images[0]

    # Resize the image to 80x40
    image = image.resize((80, 40))

    # Apply color mapping
    image = map_to_nearest_color(image)

    # Save the image
    image.save(output_path)
    print(f"Image saved to {output_path}")

if __name__ == "__main__":
    base_path = "./ziped/unet"  # Path to the directory containing your model files
    output_path = "generated_image.png"
    generate_single_image(base_path, output_path,random.randint(0,129999999))

  0%|          | 0/1000 [00:00<?, ?it/s]

Image saved to generated_image.png


In [ ]:
import base64
import numpy as np
from PIL import Image, ImageOps
from keras.models import load_model



app = FastAPI()
origins = ["*"]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

def classify_image(image_path):
    # Load the model and labels
    model = load_model("./keras_model.h5", compile=False)
    class_names = open("./labels.txt", "r").readlines()

    # Prepare the image
    image = Image.open(image_path).convert("RGB")
    image = ImageOps.fit(image, (224, 224), Image.Resampling.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    data[0] = normalized_image_array

    # Predict and get the best result
    prediction = model.predict(data)
    index = np.argmax(prediction)

    return class_names[index][2:].strip()

@app.get('/imagen')
def imageGeneration():

    base_path = "./ziped/unet"  # Path to the directory containing your model files
    output_path = "generated_image.png"
    generate_single_image(base_path, output_path,random.randint(0,129999999))

    with open(output_path, "rb") as image_file:
      encoded_string = base64.b64encode(image_file.read())


    result = classify_image(output_path)


    return {
        "bs64": encoded_string,
        "type": result
    }
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://27fa-104-196-242-9.ngrok-free.app


INFO:     Started server process [203]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


  0%|          | 0/1000 [00:00<?, ?it/s]

Image saved to generated_image.png


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-88c0c189a478>", line 25, in <cell line: 25>
    uvicorn.run(app, port=8000)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 577, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 65, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/

...

  0%|          | 0/1000 [00:00<?, ?it/s]

Image saved to generated_image.png
1/1 [==============================] - 2s 2s/step
INFO:     2607:f140:400:20:f00f:2d68:f4de:7a7f:0 - "GET /imagen HTTP/1.1" 200 OK


  0%|          | 0/1000 [00:00<?, ?it/s]

Image saved to generated_image.png


1/1 [==============================] - 1s 726ms/step
INFO:     2607:f140:400:20:f00f:2d68:f4de:7a7f:0 - "GET /imagen HTTP/1.1" 200 OK
